In [ ]:
import pandas as pd
import os 
import nltk


## Fonctions importantes pour le nettoyage 

In [ ]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)  # enlever URLs
    text = re.sub(r"[^a-zA-Zàâçéèêëîïôûùüÿñæœ\s]", " ", text)  # enlever ponctuation/chiffres
    return text

### Fonction qui suppriment les stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
x = [
    "abord", "afin", "ah", "aie", "ainsi", "allo", "allô", "alors", "après", "assez", "attendu", "aucun", "aucune", "aucuns",
    "aujourd", "aujourd'hui", "auquel", "aura", "auront", "aussi", "autre", "autres", "aux", "auxquelles", "auxquels", "avaient",
    "avais", "avait", "avant", "avec", "avoir", "avons", "ayant", "bah", "bas", "beaucoup", "bien", "c", "c'est", "ça", "car",
    "ce", "ceci", "cela", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "chaque", "chez", "chose", "ci", "comme", "comment",
    "contre", "couic", "d", "dans", "de", "debout", "dedans", "dehors", "def", "depuis", "derrière", "des", "désormais", "desquelles",
    "desquels", "dessous", "dessus", "deux", "deuxième", "devant", "devers", "devra", "devront", "difficile", "dire", "dit", "dite",
    "dits", "divers", "diverse", "diverses", "doit", "doivent", "donc", "dont", "dos", "d'autres", "du", "duquel", "duquelles",
    "duquels", "durant", "e", "eh", "elle", "elle-même", "elles", "elles-mêmes", "en", "encore", "entre", "envers", "environ",
    "es", "est", "est-ce", "et", "etant", "etc", "étaient", "étais", "était", "étant", "être", "eu", "eue", "eues", "eux", "eux-mêmes",
    "excepté", "existe", "existe-t-il", "f", "faut", "faut-il", "femme", "femmes", "feront", "fête", "fêtes", "fin", "fini", "finir", "finit",
    "finitive", "finitives", "fois", "font", "forcément", "fort", "forte", "fortes", "forts", "fou", "fouiller", "fouillé", "fouillée",
    "fouillés", "fous", "fut", "fut-ce", "fut-il", "fut-elle", "fut-on", "fut-ils","si","ici","où","là","très","parce","va","ca","oui","non","tout","toute"
] + list("abcdefghijklmnopqrstuvwxyz") 
stop_words = set(stopwords.words('french'))
stop_words = stop_words | set(x)
def remove_stopwords(text):
    words = text.split()
    words_clean = [w for w in words if w.lower() not in stop_words]
    return " ".join(words_clean)

 ### Load Datasets

In [ ]:

# chemins vers dossiers, à modifier en fonction de l'environnement de travail
folder1 = "scraped_messages_1509/"
folder2 = "scraped_messages_1609/"
output_folder = "merged_messages1"

os.makedirs(output_folder, exist_ok=True)

# on récupère la liste des fichiers CSV du premier dossier
for filename in os.listdir(folder1):
    if filename.endswith(".csv") and filename in os.listdir(folder2):
        file1 = os.path.join(folder1, filename)
        file2 = os.path.join(folder2, filename)

        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(file2)

        merged = pd.concat([df1, df2], ignore_index=True)
        # suppression des doublons sur message_id & date
        merged = merged.drop_duplicates(subset=["message_id", "date"])
        merged = merged.sort_values(by = "date")

        output_file = os.path.join(output_folder, filename)
        merged.to_csv(output_file, index=False)
        print(f"Fichier fusionné sauvegardé : {output_file}")

In [ ]:
# dossier contenant les CSV fusionnés
input_folder = "merged_messages1/"
output_file = "all_merged_messages.csv"

# liste des fichiers CSV
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# concaténation de tous les CSV
dfs = []
for file in csv_files:
    path = os.path.join(input_folder, file)
    df = pd.read_csv(path)
    dfs.append(df)

# fusion finale
final_merged = pd.concat(dfs, ignore_index=True)

# suppression des doublons (au cas où)
final_merged = final_merged.drop_duplicates(subset=["message_id", "date"])
final_merged = final_merged.sort_values(by="date")

# sauvegarde
final_merged.to_csv(output_file, index=False)

print(f"✅ Tous les fichiers ont été fusionnés dans : {output_file}")

 #### Applications des fonctions créées tout à l'heure 

In [ ]:
data["clean"] = data["text"].apply(clean_text)
data["clean_stopwords"] = data["clean"].apply(remove_stopwords)

In [ ]:
from collections import Counter

all_words = " ".join(data["clean_stopwords"]).split()
counter = Counter(all_words)

# 20 mots les plus fréquents
print(counter.most_common(20))


#### On gère les lignes qui n'ont que très peu de mots (souvent des non-sens comme "jzzjz")

In [ ]:
data = data[data["clean_stopword"].str.split().str.len() >= 3].copy()

## Début de l'analyse du texte

## Ici on combine TfIDF à LDA (Latent Dirichlet Allocation) 
TfIDF est une méthode de vectorisation, le texte étant par définition une suite de caractères, il faut trouver un moyen de le numériser tout en gardant le contexte et la fréquence d'apparitions des termes. TfIDF prend un mot dans un Document( pour nous un message = un Document) et retourne sa fréquence dans le corpus suivant la fonction : $$ Tf_{ij} = \frac{f_{t,d}}{\sum_{t'\in d}f_{t',d}}$$ où $f_{t,d}$ correspond à la fréquence d'apparition du mot $t$ dans le document $d$. La fréquence inverse est donnée par $$Idf_i = log(\frac{|D|}{|d, t_i \in d|}), \text{avec |D| le nombre total de messages}$$. Au final TfIDF est donnée par $$ TfIDF_{ij} = Tf_{ij}\cdot Idf_i$$

#### LDA
L'algorithme LDA (Latent Dirichlet Allocation) permet d'analyser un corpus de document et permet de faire de la classification rapidement. L'algorithme divise en "topics" le document et assigne à chaque mot un topic. (Voir le papier original pour l'implémentation, je n'ai pas eu le temps de regarder !)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texts = data['clean_stopword'].dropna().astype(str) #remove NAs

vectorizer = TfidfVectorizer(max_df=0.9, min_df=5) #les paramètres permettent de supprimer des termes trop/pas assez fréquents de l'analyse
X = vectorizer.fit_transform(texts)
feature_names = vectorizer.get_feature_names_out()

# =========================
#  Modèle LDA
# =========================
n_topics = 5 
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# =========================
# =========================
no_top_words = 10
for topic_idx, topic in enumerate(lda.components_):
    print(f"\nTopic {topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# =========================
# =========================
topic_values = lda.transform(X)
data['dominant_topic'] = topic_values.argmax(axis=1)

# =========================
#  WordCloud par topic
# =========================
for topic_idx, topic in enumerate(lda.components_):
    topic_words = {feature_names[i]: topic[i] for i in topic.argsort()[:-50 - 1:-1]}  # top 50 mots
    wc = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(topic_words)

    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"WordCloud Topic {topic_idx}")
    plt.show()


## Embedding
C'est une autre méthode de vectorisation, qui garde le contexte et le sens des mots, mais requiert bien plus de calculs

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
# -------------------------------
# MiniLM est rapide et léger (384 dimensions)
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# -------------------------------
# 4. Calcul des embeddings par batch
# -------------------------------
batch_size = 5000
embeddings = []

for start_idx in tqdm(range(0, len(data), batch_size)):
    end_idx = min(start_idx + batch_size, len(data))
    batch_texts = data['clean'].iloc[start_idx:end_idx].tolist()
    batch_embeddings = model.encode(batch_texts, show_progress_bar=False)
    embeddings.append(batch_embeddings)

# Fusionner tous les batches en un seul array
embeddings = np.vstack(embeddings)
print("Shape des embeddings :", embeddings.shape)  # (150000, 384)

In [ ]:
emb_df = pd.concat([data.reset_index(drop=True), pd.DataFrame(embeddings)], axis=1)
table = pa.Table.from_pandas(emb_df)
pq.write_table(table, "messages_with_embeddings.parquet")

print("Embeddings calculés et stockés avec succès !")

#### Lire le fichier parquet

In [ ]:

df_emb = pd.read_parquet("messages_with_embeddings.parquet",engine = "fastparquet")
print(df_emb.head())

In [ ]:
columns_list = [  #Liste qui contient chaque colonne qui n'est pas une des dimensions du embedding (à faire avant avec pd.col)
    "chat",
    "chat_name",
    "message_id",
    "date",
    "sender_id",
    "text",
    "reply_to",
    "clean",
    "clean_stopword",
    "nb_mots_champ",
    "somme_mots",
    "nb_champ_relatif",
    "clean_no_stop"
]
  # adapter selon tes colonnes
embedding_columns = df_emb.columns.difference(columns_list)

X = df_emb[embedding_columns].values

# Normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## clustering par K-means
from sklearn.cluster import KMeans

n_clusters = 10  # tu peux ajuster
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_emb['cluster'] = kmeans.fit_predict(X_scaled)

print(df_emb[['text', 'cluster']].head())



### Representation graphique du clustering, problème étant que l'on joue avec des données de 380 dimensions.
On utilise le package umap

In [ ]:
import umap.umap_ as umap

reducer = umap.UMAP(n_components=2, random_state=42,unique = True)
X_umap = reducer.fit_transform(X_scaled)

import matplotlib.pyplot as plt
import seaborn as sns

reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_scaled)

df_emb['x'] = X_umap[:,0]
df_emb['y'] = X_umap[:,1]

plt.figure(figsize=(10,6))
sns.scatterplot(data=df_emb, x='x', y='y', hue='cluster', palette='tab10', s=50, alpha=0.8)
plt.title("Clustering des messages")
plt.show()
